<div class="alert alert-success">  

-------
# XArray 101 🌍  
-------
* Jupyter and Python Basics
* Xarray Intro
* Xarray Advanced
* Vector Data
* Remote Sensing
* Visualization
* __Excercise__

-------  
</div>

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# extra lib to install
! conda install -c conda-forge pyepsg

In [ ]:
import intake

import cartopy.crs as ccrs
import geopandas as gpd
import holoviews as hv
import numpy as np
import xarray as xr
import rioxarray
from pathlib import Path

import hvplot.pandas
import hvplot.xarray

# Optional: Build a data catalog

## Attempt one: list all assets individually

In [ ]:
%%writefile catalog_brunnenkopf_type1.yml

# exclosures - non-grazed
# grazed ref - grazed
# fenced - excluded area (spring, river)

sources: 

    exclosure:
        description: 'Area excluded from grazing'
        driver: shapefile
        args:
            urlpath: 'data_anne/VectorData/Kontrollflaechen_etc_EPSG_25832/Exclosures.shp' 
    
    grazed:
        description: 'Area grazed'
        driver: shapefile
        args:
            urlpath: 'data_anne/VectorData/Kontrollflaechen_etc_EPSG_25832/Grazed_Reference.shp' 

    fenced:
        description: 'Area fenced and excluded from analysis'
        driver: shapefile
        args:
            urlpath: 'data_anne/VectorData/Kontrollflaechen_etc_EPSG_25832/Fenced_area.shp' 


In [ ]:
cat1 = intake.open_catalog("catalog_brunnenkopf_type1.yml")
list(cat1)

In [ ]:
cat1.exclosure.describe()

In [ ]:
# read data into geopandas dataframes
nongrazed = cat1.exclosure.read()
grazed = cat1.grazed.read()
fenced = cat1.fenced.read()

## Attempt two: use parameters to differentiate between assets

In [ ]:
%%writefile catalog_brunnenkopf_type2.yml

# exclosures - non-grazed
# grazed ref - grazed
# fenced - excluded area (spring, river)

sources: 

    brunnenkopf:
        description: 'Brunnenkopf Alm Vector Data'
        driver: shapefile
        parameters:
            area_type:
                description: 'Type of area: Exclosures, Fenced_area, or Grazed_Reference'
                type: str
                default: "Exclosures"
                allowed: ["Exclosures", "Fenced_area", "Grazed_Reference"]
        args:
            urlpath: 'data_anne/VectorData/Kontrollflaechen_etc_EPSG_25832/{{area_type}}.shp' 


In [ ]:
cat2 = intake.open_catalog("catalog_brunnenkopf_type2.yml")
list(cat2)

In [ ]:
cat2.brunnenkopf.describe()

In [ ]:
# read data into geopandas dataframes
nongrazed = cat2.brunnenkopf(area_type="Exclosures").read()
grazed = cat2.brunnenkopf(area_type="Grazed_Reference").read()
fenced = cat2.brunnenkopf(area_type="Fenced_area").read()

# Inspect the data

## Vector data

We use `hvplot` (and in the background `holoviews` and `geoviews`) for a quick interactive plot of the experimental setup at Brunnenkopf.

⚠️ **NOTE** ⚠️ 
- Tiles can be used to get our data some context or to prettify the maps. There are a bunch of options and some work better for specify map settings (also, 
not all tile sources provide data for any zoom level)
> Valid tiles sources: 'CartoDark', 'CartoEco', 'CartoLight', 'CartoMidnight', 'EsriImagery', 'EsriNatGeo', 'EsriReference', 'EsriTerrain', 
> 'EsriUSATopo', 'OSM', 'StamenLabels', 'StamenTerrain', 'StamenTerrainRetina', 'StamenToner', 'StamenTonerBackground', 
> 'StamenWatercolor', 'Wikipedia'
- The data is projected in [UTM 32N](https://epsg.io/25832)
- Map tiles are in epsg:4326. In order to overlay our data correctly we either project the geopandas dataframe on the fly (using `.to_epsg(4326)`) or better we specify a projection of the original data so hvplot can reproject it for us automatically (if you provide the `crs` argument you don't have to provide `geo=True`).

In [ ]:
# data/ source projection UTM 32N
projection = ccrs.epsg(25832)

# we add the tiles to the first/ base layer of our plot
plot1 = nongrazed.hvplot(crs=projection, label="Exclosure", tiles="OSM")
plot2 = fenced.hvplot(crs=projection, label="Fenced", fill_color="none", line_color="red", line_dash="dashed", line_width=2)
plot3 = grazed.hvplot(crs=projection, label="Grazed (Ref)", fill_color="none", line_color="blue", line_dash="dotted", line_width=2)

# combine the layers into one (interactive) plot
(plot1 * plot2 * plot3).opts( width=800, height=600, title="Brunnenkopf Setup")

## Multi-spectral raster data

In [ ]:
%%writefile catalog_brunnenkopf_msp.yml

sources: 

    msp_2019:
        description: 'Brunnenkopf MutiSpectral Data 2019'
        driver: rasterio
        parameters:
            band:
                description: 'Spectral band'
                type: str
                default: "red"
                allowed: ["red", "green", "nir", "red edge"]
        args:
            urlpath: 'data_anne/MultispectralData/brunnenkopf_msp_20190716/brunnenkopf_msp_20190716_transparent_reflectance_{{ band }}.tif' 
            chunks: {}

    msp_2020:
        description: 'Brunnenkopf MutiSpectral Data 2020'
        driver: xarray
        parameters:
            band:
                description: 'Spectral band'
                type: str
                default: "red"
                allowed: ["red", "green", "nir", "red edge"]
        args:
            urlpath: 'data_anne/MultispectralData/brunnenkopf_msp_20200728/brunnenkopf_msp_20200728_transparent_reflectance_{{ band }}.tif' 
            chunks: {}


In [ ]:
cat_msp = intake.open_catalog("catalog_brunnenkopf_msp.yml")
list(cat_msp)

In [ ]:
cat_msp.msp_2019.describe()

In [ ]:
red = cat_msp.msp_2019(band='red').read()
red = red.where(red>-10000).squeeze(drop=True) 
red = red / red.max()
red.hvplot(aspect=1, cmap='reds')

In [ ]:
nir = cat_msp.msp_2019(band='nir').read()
nir = nir.where(nir>-10000).squeeze(drop=True) 
nir = nir / nir.max()
nir.hvplot(aspect=1, cmap='blues')

In [ ]:
def calc_ndvi(nir, red):
    """NDVI"""
    return (nir - red) / (nir + red)

In [ ]:
ndvi = calc_ndvi(nir, red)
ndvi

In [ ]:
p1 = nongrazed.hvplot(crs=projection, label="Non-Grazed", fill_color="none", line_color="red", line_dash="dotted", line_width=2)
p2 = grazed.hvplot(crs=projection, label="Grazed", fill_color="none", line_color="blue", line_dash="dotted", line_width=2)

In [ ]:
ndvi.hvplot(crs=projection, aspect=1, cmap='BrBG') * p1 * p2

### Calculate stats

In [ ]:
ndvi = ndvi.rio.set_crs(nongrazed.crs)
nongrazed_ndvi = ndvi.rio.clip(nongrazed.geometry)
print(f"NDVI non-grazed: {nongrazed_ndvi.mean().values:.2f} ± {nongrazed_ndvi.std().values:.2f}")

grazed_ndvi = ndvi.rio.clip(grazed.geometry)
grazed_ndvi.mean().values
print(f"NDVI grazed:     {grazed_ndvi.mean().values:.2f} ± {grazed_ndvi.std().values:.2f}")


# RGB Images

In [ ]:
rgbdata = Path("data_anne/RGBData")

rgb2020 = xr.open_rasterio(rgbdata / "brunnenkopf_rgb_20200728_rs.tif", chunks={'band':4, 'x':1024, 'y':1024})
rgb2020

In [ ]:
rgb2018 = xr.open_rasterio(rgbdata / "brunnenkopf_rgb_20180718_rs.tif", chunks={'band':4, 'x':1024, 'y':1024})
rgb2019 = xr.open_rasterio(rgbdata / "brunnenkopf_rgb_20190716_rs.tif", chunks={'band':4, 'x':1024, 'y':1024})

The data is huge, let's select a subset and check this out first

In [ ]:
a = rgb2020.isel(x=slice(12000, 15000),  y=slice(7000, 10000))
x_slice = slice(a.x.min().values, a.x.max().values)
y_slice = slice(a.y.max().values, a.y.min().values) # NOTE the negative orientation!

In [ ]:
# show a (random) subset of 3000x3000 px...
r2018 = rgb2018.sel(x=x_slice, y=y_slice).hvplot(aspect=1, title="2018")
r2019 = rgb2019.sel(x=x_slice, y=y_slice).hvplot(aspect=1, title="2019")
r2020 = rgb2020.sel(x=x_slice, y=y_slice).hvplot(aspect=1, title="2020")
r2018 + r2019 + r2020

In [ ]:
grazed_ref

In [ ]:
grazed_ref.shape

In [ ]:
regions = []
for index, row in grazed_ref.iterrows():
    b = rgb2018.rio.clip([row.geometry], drop=True)
    regions.append( b.persist().to_dataset(name='z') )
    
hv.Layout([r.hvplot.rgb(x='x', y='y', z='z', bands='band', aspect=1).opts(axiswise=True) for r in regions]).cols(3)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
regions = []
for _, region in grazed_ref.iterrows():   
    b = rgb2018.rio.clip([region.geometry], drop=False, invert=True)
    regions.append(b)

print(regions)
    
fig, ax = plt.subplots(1, grazed_ref.shape[0], figsize=(3*grazed_ref.shape[0], 4))

for i, p in enumerate(regions):
    regions[i].plot(ax=ax[i])